# Digital Twin Brain Model – Perfusion & Ischemia Simulation
This notebook visualizes simulated brain perfusion data, flags ischemia risk regions, and extends the model with a basic time-series ischemia progression simulation.

## Step 1: Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Step 2: Load the Dataset

In [ ]:
df = pd.read_csv('../data/brain_perfusion_mock.csv')
df.head()

## Step 3: Visualize Metrics as a Grouped Bar Chart

In [ ]:

x = np.arange(len(df["Region"]))
perf = df["PerfusionRate"]
ox = df["OxygenSaturation"]
viability = df["TissueViabilityScore"] * 100

fig, ax = plt.subplots(figsize=(10, 4))
bar_width = 0.25
ax.bar(x - bar_width, perf, width=bar_width, label="Perfusion", color="skyblue")
ax.bar(x, ox, width=bar_width, label="Oxygen Saturation", color="lightgreen")
ax.bar(x + bar_width, viability, width=bar_width, label="Viability (0–100)", color="salmon")

ax.set_xticks(x)
ax.set_xticklabels(df["Region"])
ax.set_ylabel("Value")
ax.set_title("Brain Region Metrics")
ax.legend()
plt.tight_layout()
plt.show()


## Step 4: Flag Regions at Risk of Ischemia

In [ ]:

df['IschemiaRisk'] = (df['PerfusionRate'] < 35) & (df['TissueViabilityScore'] < 0.5)
df[['Region', 'PerfusionRate', 'OxygenSaturation', 'TissueViabilityScore', 'IschemiaRisk']]


## Step 5: Time-Series Simulation of Ischemia Progression
We simulate the degradation of tissue viability over 10 minutes assuming constant hypoperfusion.

In [ ]:

time_steps = 10
viability_over_time = []

# assume 5% drop per minute for high-risk regions, 1% otherwise
for t in range(time_steps):
    drop = np.where(df["IschemiaRisk"], 0.05, 0.01)
    new_viability = np.maximum(df["TissueViabilityScore"] - drop * t, 0)
    viability_over_time.append(new_viability)

viability_over_time = np.array(viability_over_time)

# Plot progression
plt.figure(figsize=(12, 6))
for i in range(len(df)):
    plt.plot(range(time_steps), viability_over_time[:, i], label=df["Region"][i])
plt.xlabel("Time (minutes)")
plt.ylabel("Tissue Viability Score")
plt.title("Simulated Ischemia Progression Over Time")
plt.legend(loc="upper right", ncol=2)
plt.grid(True)
plt.tight_layout()
plt.show()


## Step 6: Add Predictive Flags and Brain Death Alerts

In [ ]:

# Define thresholds
BRAIN_DEATH_THRESHOLD = 0.2

# Determine if and when each region crosses the threshold
brain_death_times = []

for i in range(viability_over_time.shape[1]):
    below_thresh = np.where(viability_over_time[:, i] < BRAIN_DEATH_THRESHOLD)[0]
    if len(below_thresh) > 0:
        brain_death_times.append(below_thresh[0])
    else:
        brain_death_times.append(None)

df["BrainDeathMinute"] = brain_death_times
df[["Region", "IschemiaRisk", "BrainDeathMinute"]]


## Step 7: Simulate Response to Treatment (Restored Perfusion at Minute 5)
Assume treatment is given at minute 5, stopping further decline in viability.

In [ ]:

# Recalculate viability with intervention
viability_treated = []

for t in range(time_steps):
    if t < 5:
        drop = np.where(df["IschemiaRisk"], 0.05, 0.01)
    else:
        drop = np.zeros_like(df["TissueViabilityScore"])
    new_viability = np.maximum(df["TissueViabilityScore"] - drop * t, 0)
    viability_treated.append(new_viability)

viability_treated = np.array(viability_treated)

# Plot both untreated and treated
plt.figure(figsize=(12, 6))
for i in range(len(df)):
    plt.plot(range(time_steps), viability_over_time[:, i], linestyle="--", label=f"{df['Region'][i]} Untreated")
    plt.plot(range(time_steps), viability_treated[:, i], linestyle="-", label=f"{df['Region'][i]} Treated")
plt.xlabel("Time (minutes)")
plt.ylabel("Tissue Viability Score")
plt.title("Viability Over Time: Treated vs Untreated Regions")
plt.grid(True)
plt.legend(ncol=2, fontsize=8)
plt.tight_layout()
plt.show()
